In [1]:
using Printf: @printf, @sprintf
import JSON
using DataFrames: nrow
using Dates: Day
using BenchmarkTools

In [2]:
using Revise
import TempModel
using GaussianProcesses

┌ Info: Recompiling stale cache file /Users/imolk/Library/Julia/alternative_depots/climate/compiled/v1.1/TempModel/5V5SX.ji for TempModel [429a4ede-8e99-57b3-891e-c3971593d88c]
└ @ Base loading.jl:1184


In [3]:
ICAO = "KBDL"
GPmodel = "matern"
data_dir= "../data"
save_dir= "../../saved"
k_nearest = 5
crossval = true

if GPmodel=="fixed_var"
    k_spatiotemporal,logNoise = TempModel.fitted_sptemp_fixedvar()
elseif GPmodel=="free_var"
    k_spatiotemporal,logNoise = TempModel.fitted_sptemp_freevar()
elseif GPmodel=="sumprod"
    k_spatiotemporal,logNoise = TempModel.fitted_sptemp_sumprod()
elseif GPmodel=="SExSE"
    k_spatiotemporal,logNoise = TempModel.fitted_sptemp_SExSE()
elseif GPmodel=="diurnal"
    k_spatiotemporal,logNoise = TempModel.fitted_sptemp_diurnal()
elseif GPmodel=="simpler"
    k_spatiotemporal,logNoise = TempModel.fitted_sptemp_simpler()
elseif GPmodel=="matern"
    kdict = TempModel.kernel_sptemp_matern(;kmean=true)
    k_spatiotemporal = kdict[:spatiotemporal]
    logNoise = -1.0
else
    error(@sprintf("unknown model: %s", GPmodel))
end

epsg = 3857 # Web Mercator (m)
isdList = TempModel.read_isdList(; data_dir=data_dir, epsg=epsg)
isd_wData = TempModel.stations_with_data(isdList; data_dir=data_dir)

test_station = isd_wData[isd_wData[:ICAO].==ICAO, :]
@assert nrow(test_station) == 1
USAF = test_station[1, :USAF]
WBAN = test_station[1, :WBAN]

isd_nearest_and_test = TempModel.find_nearest(isd_wData, USAF, WBAN, k_nearest)
isd_nearest = isd_nearest_and_test[2:end,:]

,USAF,WBAN,NAME,CTRY,STATE,ICAO,LAT,LON,ELEV,BEGIN,END,X_PRJ,Y_PRJ
,Int64⍰,Int64⍰,String⍰,String⍰,String⍰,String⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Float64,Float64
1,744910,14703,WESTOVER AFB/METROPOLITAN AIRPORT,US,MA,KCEF,42.2,-72.533,73.5,1941,2015,-8.07434e6,5.19099e6
2,725040,94702,IGOR I SIKORSKY MEMORIAL AIRPORT,US,CT,KBDR,41.158,-73.129,1.5,1942,2015,-8.14068e6,5.03567e6
3,725070,14765,THEODORE F GREEN STATE AIRPORT,US,RI,KPVD,41.722,-71.433,18.3,1942,2015,-7.95189e6,5.11943e6
4,725038,14714,STEWART INTERNATIONAL AIRPORT,US,NY,KSWF,41.5,-74.1,149.7,1942,2015,-8.24877e6,5.08637e6
5,725180,14735,ALBANY INTERNATIONAL AIRPORT,US,NY,KALB,42.743,-73.809,95.1,1946,2015,-8.21638e6,5.27294e6


In [4]:
hourly_data = TempModel.read_Stations(isd_nearest; data_dir=data_dir)
size(hourly_data)

(57710, 10)

In [5]:
using TempModel: make_chunks_and_folds

In [6]:
reals, folds_reals = make_chunks_and_folds(k_spatiotemporal, logNoise, 
        isd_nearest_and_test, hourly_data; window=Day(5))
;

creating GP chunks


In [7]:
print([r.nobs for r in reals.reals])

[861, 695, 666, 679, 726, 961, 823, 858, 680, 875, 734, 710, 776, 700, 626, 738, 746, 772, 791, 965, 699, 744, 803, 594, 589, 716, 692, 803, 583, 752, 994, 681, 617, 889, 776, 812, 699, 857, 813, 743, 613, 867, 799, 775, 746, 911, 935, 949, 916, 800, 939, 935, 813, 1095, 985, 651, 1070, 698, 656, 691, 780, 840, 706, 696, 677, 615, 759, 947, 834, 770, 799, 1202, 1003]

In [65]:
let
    gp = reals.reals[end-1] # the biggest one
    buffer = Matrix{Float64}(undef, gp.nobs, gp.nobs)
    @time GaussianProcesses.update_mll_and_dmll!(gp, buffer; domean=false, kern=true, noise=true)
end
;

  0.789442 seconds (22 allocations: 29.703 KiB)


In [78]:
let
    gp = reals.reals[end-1] # the biggest one
    folds = folds_reals[end-1]
    buffer = Matrix{Float64}(undef, gp.nobs, gp.nobs)
    @time GaussianProcesses.dlogpdθ_CVfold(gp, folds; domean=false, kern=true, noise=true)
end
;

  1.954698 seconds (1.58 k allocations: 254.084 MiB, 9.04% gc time)


In [79]:
using Profile
Profile.clear()
let
    gp = reals.reals[end-1] # the biggest one
    folds = folds_reals[end-1]
    buffer = Matrix{Float64}(undef, gp.nobs, gp.nobs)
    GaussianProcesses.dlogpdθ_CVfold(gp, folds; domean=false, kern=true, noise=true);
    @profile GaussianProcesses.dlogpdθ_CVfold(gp, folds; domean=false, kern=true, noise=true);
#     @profile GaussianProcesses.dlogpdθ_CVfold(gp, folds; domean=false, kern=true, noise=false);
end
;

In [80]:
Profile.print(maxdepth=10, mincount=50)

622 ./task.jl:259; (::getfield(IJulia, Symbol("##15#1...
 622 ...4UizY/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  622 ./essentials.jl:741; invokelatest
   622 ./essentials.jl:742; #invokelatest#1
    622 ...rc/execute_request.jl:67; execute_request(::ZMQ.Socket, ::I...
     621 ...c/SoftGlobalScope.jl:218; softscope_include_string(::Modu...
      621 ./boot.jl:328; eval
       621 ./none:0; (::getfield(GaussianProcesses, ...
        602 .../crossvalidation.jl:292; #dlogpdθ_CVfold#90(::Bool, ::B...
         337 ...crossvalidation.jl:230; dlogpdθ_CVfold_kern!(::SubArr...
          337 ...nels/sum_kernel.jl:64; grad_slice!
         201 ...crossvalidation.jl:240; dlogpdθ_CVfold_kern!(::SubArr...
          80 ./array.jl:308; gradient_fold
          51 ...crossvalidation.jl:203; gradient_fold


In [24]:
Profile.print(maxdepth=10, mincount=50)

563 ./task.jl:259; (::getfield(IJulia, Symbol("##15#1...
 563 ...4UizY/src/eventloop.jl:8; eventloop(::ZMQ.Socket)
  563 ./essentials.jl:741; invokelatest
   563 ./essentials.jl:742; #invokelatest#1
    563 ...rc/execute_request.jl:67; execute_request(::ZMQ.Socket, ::I...
     563 ...c/SoftGlobalScope.jl:218; softscope_include_string(::Modu...
      563 ./boot.jl:328; eval
       563 ./none:0; (::getfield(GaussianProcesses, ...
        549 .../crossvalidation.jl:293; #dlogpdθ_CVfold#85(::Bool, ::B...
         227 ...crossvalidation.jl:217; dlogpdθ_CVfold_kern!(::SubArr...
          227 ...nels/sum_kernel.jl:64; grad_slice!
         132 ...crossvalidation.jl:237; dlogpdθ_CVfold_kern!(::SubArr...
          132 ...Kouno/src/pdmat.jl:48; inv(::PDMats.PDMat{Float64,Ar...
         68  ...nels/sum_kernel.jl:66; dlogpdθ_CVfold_kern!(::SubArr...
          52 ...kernels/kernels.jl:117; grad_slice!(::Array{Float64,2...


So cross-validation is about 5 times slower than marginal likelihood.
Not exactly the end of the world, but annoying nonetheless.
What's annoying is that half of that time (1080/2241, or is it (1080+848)/2241?) is spent computing computing the gradients of the covariance matrix, which is also done by `update_mll_and_dmll!`.
The difference is that `update_mll_and_dmll!` does so online, while for cross-validation it's stored in a buffer matrix. It's also where all the memory allocations occur.
This is infuriating, why is so much memory being allocated?

In [12]:
let
    gp = reals.reals[end-1] # the biggest one
    folds = folds_reals[end-1]
    buffer = Matrix{Float64}(undef, gp.nobs, gp.nobs)
    @time for j in 1:GaussianProcesses.num_params(gp.kernel)
        @time GaussianProcesses.grad_slice!(buffer, gp.kernel, gp.x, gp.data, j)
    end
end

  0.056489 seconds (178 allocations: 11.453 KiB)
  0.038927 seconds
  0.038929 seconds
  0.039076 seconds
  0.038072 seconds
  0.043813 seconds
  0.034331 seconds
  0.040935 seconds
  0.034591 seconds
  0.041859 seconds
  0.034664 seconds
  0.042651 seconds
  0.035812 seconds
  0.046988 seconds
  0.037097 seconds
  0.614490 seconds (3.47 k allocations: 136.594 KiB)


In [86]:
@time opt_out = TempModel.optim_kernel(k_spatiotemporal, logNoise, isd_nearest, hourly_data, :Optim; 
                        window=Day(8), x_tol=1e-4, f_tol=1e-4);
;

creating GP chunks
begin optimization


InterruptException: InterruptException: